## Pagerank Algorithm

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix, csr_matrix
from numpy.linalg import norm
import time
#np.seterr(divide='ignore')

### Data

In [8]:
def read_data(filename):
    # read data
    data = pd.read_csv(filename, sep='\t', header=None, names=['from', 'to','weight'])
    # reduce from and to values by 1 so that nodes start from 0 (for easier calculations)
    data['from'] = data['from']-1
    data['to'] = data['to']-1
    
    # number of edges
    edges = len(data)    
    
    # dict{webpage:number of times webpage was a "from" page}
    out_dict = data.groupby('from')['to'].count().to_dict()
    # dict{webpage: list of webpages that lead to this webpage}
    in_dict = data.groupby('to')['from'].aggregate(lambda x: list(x)).to_dict()
    

    # number of nodes
    nodes = len(set(out_dict) | set(in_dict))
              
    # update in_dict with unseen nodes
    for node in range(nodes):  
        if node not in in_dict.keys():
             in_dict[node] = []  
    
    # connectivity sparse matrix
    sparse_m = csr_matrix(([1]*edges,(data['to'].tolist(), data['from'].tolist())), shape=(nodes, nodes))
                
    return in_dict, out_dict, nodes, edges, sparse_m


#     D = sparse.lil_matrix((nodes, 1))
#     for key in dict_out.keys():
#         D[key] = 1/dict_out[key]


In [9]:
ind, outd, nodes, edges, sp_m = read_data('stanweb.dat\stanweb.dat')

### Algorithms

In [ ]:
import numpy as np
from scipy.sparse import csc_matrix

def pagerank_power(G, s = .85, maxerr = .0001):
    """
    Computes the pagerank for each of the n states
    Parameters
    ----------
    G: matrix representing state transitions
       Gij is a binary value representing a transition from state i to j.
    s: probability of following a transition. 1-s probability of teleporting
       to another state.
    maxerr: if the sum of pageranks between iterations is bellow this we will
            have converged.
    """
    n = G.shape[0]

    # transform G into markov matrix A
    A = csc_matrix(G,dtype=np.float)
    rsums = np.array(A.sum(1))[:,0]
    ri, ci = A.nonzero()
    A.data /= rsums[ri]

    # bool array of sink states
    sink = rsums==0

    # Compute pagerank r until we converge
    ro, r = np.zeros(n), np.ones(n)
    while np.sum(np.abs(r-ro)) > maxerr:
        ro = r.copy()
        # calculate each pagerank at a time
        for i in xrange(0,n):
            # inlinks of state i
            Ai = np.array(A[:,i].todense())[:,0]
            # account for sink states
            Di = sink / float(n)
            # account for teleportation to state i
            Ei = np.ones(n) / float(n)

            r[i] = ro.dot( Ai*s + Di*s + Ei*(1-s) )

    # return normalized pagerank
    return r/float(sum(r))

In [ ]:
def pagerank_galgo(G, alpha=0.85, tol=10**-8):

    n = G.shape[0]

    out_alpha = G.sum(axis=0).T/alpha
    r = np.ones((n,1))/n
    error = 1
    t = 0
    while error > tol:
        t += 1
        r_new = G.dot((r/out_alpha))
        #  L≤alpha due to dead-ends
        L = r_new.sum()
        # re-insert leaked page-rank
        # now all ranks add to one
        r_new += (1-L)/n
        # compute the error as the euclidean norm of the
        # previous ranks and the new ranks
        error = np.linalg.norm(r-r_new)
        # store the new ranks as the ranks of the current
        # iteration
        if t == 2:
            # list to return the nodes that converged from the
            # second iteration
            list_conv = []
            for i in range(r.shape[0]):
                if np.linalg.norm(r[i]-r_new[i]) < tol:
                    list_conv.append(i)
        r = r_new
    return r, t, list_conv